# Limb Vector (Zawada et. al. 2021) Comparisons

Runs the comparisons found in 

Zawada, D., Franssens, G., Loughman, R., Mikkonen, A., Rozanov, A., Emde, C., Bourassa, A., Dueck, S., Lindqvist, H., Ramon, D., Rozanov, V., Dekemper, E., Kyrölä, E., Burrows, J. P., Fussen, D., and Degenstein, D.: Systematic comparison of vectorial spherical radiative transfer models in limb scattering geometry, Atmos. Meas. Tech., 14, 3953–3972, https://doi.org/10.5194/amt-14-3953-2021, 2021.

The input data and output data for this comparison is downloaded automatically from the zenodo reference at https://zenodo.org/records/4292303.

In [ ]:
import sasktran2 as sk
import appdirs
from pathlib import Path
import shlex
import xarray as xr
import numpy as np


def rtm_comparison_file():
    data_dir = Path(appdirs.user_data_dir('sasktran2'))

    if not data_dir.exists():
        data_dir.mkdir(parents=True)

    file = data_dir.joinpath('zawada_AMT_rtm_comparison_data_v1.nc')

    if not file.exists():
        from zenodo_get import zenodo_get
        zenodo_get(shlex.split('--record 4292303 -o "{}"'.format(data_dir.as_posix())))

    return file

In [ ]:
def load_scenario(geometry_index: int, atmosphere_index: int, albedo_index: int, test_case: int,
                  altitude_spacing: float=500):
    rtm_file = rtm_comparison_file()

    geo = xr.open_dataset(rtm_file, group='geometry_data')
    anc = xr.open_dataset(rtm_file, group='ancillary_data')
    model = xr.open_dataset(rtm_file, group='model_data')

    albedo = float(model['albedo'].isel(albedo=albedo_index))
    albedo = np.ones(len(model['wavelength'])) * albedo

    geo = geo.isel(solar_condition=geometry_index)

    anc = anc.interp(altitude=np.arange(0, 100001, altitude_spacing))

    model = model.isel(solar=geometry_index, composition=atmosphere_index, albedo=albedo_index, test_case=test_case)

    model_geo = sk.Geometry1D(np.cos(float(geo.tangent_sza) * np.pi/180), 0.0, 6371000, anc.altitude.values.astype(float), sk.InterpolationMethod.LinearInterpolation,
                              sk.GeometryType.Spherical)


    viewing_geo = sk.ViewingGeometry()

    for alt in geo.tangent_altitude.values:
        viewing_geo.add_ray(sk.TangentAltitudeSolar(alt*1000, float(geo.tangent_saa)*np.pi/180, 200000, np.cos(float(geo.tangent_sza) * np.pi/180)))

    config = sk.Config()
    config.num_stokes = 3

    wavelengths_nm = anc.wavelength.values

    # Add Rayleigh
    atmo = sk.Atmosphere(model_geo, config, wavelengths_nm, calculate_derivatives=False)
    atmo.pressure_pa = anc.pressure.values
    atmo.temperature_k = anc.temperature.values

    atmo["rayleigh"] = sk.constituent.Rayleigh("manual", {"wavelength_nm": wavelengths_nm, "xs": anc["rayleigh_scattering_cross_section"].values*1e-4, "king_factor": np.ones_like(anc["rayleigh_scattering_cross_section"].values)})

    if atmosphere_index == 1:
        # Add ozone
        pass

    if atmosphere_index == 2:
        # Add aerosol
        pass

    config = sk.Config()
    config.num_stokes = 3

    return {'atmosphere': atmo, 'geometry': model_geo, 'config': config, 'result': model, 'viewinggeo': viewing_geo}


In [ ]:
scen = load_scenario(2, 0, 0, 0, 500)

engine = sk.Engine(scen["config"], scen["geometry"], scen["viewinggeo"])

radiance = engine.calculate_radiance(scen["atmosphere"])

In [ ]:
scen['result'] = scen['result'].isel(stokes=range(3))
scen['result']["sasktran2"] = (["wavelength", "altitude", "stokes"], radiance["radiance"].to_numpy())
p_diff = (scen['result']['sasktran2'] - scen['result']['mmm']) / scen['result']['mmm'] * 100

In [ ]:
for w in range(11):
    p_diff.isel(stokes=0, wavelength=w).plot(y='altitude')

In [ ]:
# array([70, 80, 80, 45, 60, 50, 10, 15, 35], dtype=int32)
# array([ 30,  60, 150,  50,  70, 130,  20,  70,  90], dtype=int32)

In [ ]:
scen['result']['radiance'].isel(model=2, stokes=2, wavelength=2).plot()
scen['result']['sasktran2'].isel(stokes=2, wavelength=2).plot()

In [ ]:
scen['result']['radiance'].isel(model=2, stokes=1, wavelength=2).plot()
scen['result']['sasktran2'].isel(stokes=1, wavelength=2).plot()